In [1]:
# Base Imports
import os
import json
import numpy as np
import pandas as pd


# WUBRG Imports
import WUBRG
from WUBRG import COLOR_ALIASES_SUPPORT, COLOR_ALIASES, COLOUR_GROUPINGS, COLOUR_GROUPS, COLORS


# set_info imports
import set_info
from set_info import SETS, FORMATS, STAT_NAMES
DATA_CACHE = dict()
PANDAS_CACHE = dict()
WINRATES = dict()
METAGAME_REPORT = dict()


# Function pointers to moved functions.
min_play_filter = set_info.min_play_filter
color_id_filter = set_info.color_id_filter
rarity_filter = set_info.rarity_filter
color_game_counts = set_info.color_game_counts
get_top = set_info.get_top
gen_metadata_dict = set_info.gen_metadata_dict
get_color_group_metadata = set_info.get_color_group_metadata
get_color_overview = set_info.get_color_overview
get_format_metagame_data = set_info.get_format_metagame_data


print(f'SETS:          {str(SETS)}')
print(f'FORMATS:       {str(FORMATS)}')

COLUMNS_TRUNC = ["Color", "Rarity", "ALSA", "# GP", "GP WR", "# GIH", "GIH WR"]

WRANING: 'discord' failed to import. Emoji support unavailable
SETS:          ['MID']
FORMATS:       ['PremierDraft', 'TradDraft']


In [2]:
s = 'MID'
f = 'PremierDraft'

In [3]:
# # Filters out cards that have a number of games played less than p% of # GP.
# def min_play_filter(df, p):
#     min_game = df['# GP'].sum() * (p/100)
#     return df[df['# GP'] >= min_game]

# # Filters out cards that don't fall within the colour identy.
# def color_id_filter(df, color_string):
#     if color_string is not None and color_string != '':
#         return df[df['Color'].apply(lambda x: set(x) <= set(color_string))]
#     else:
#         return df

# # Filters out cards that aren't in the given rarities.
# def rarity_filter(df, rarity):
#     if rarity is not None and rarity != '':
#         return df[df['Rarity'].apply(lambda x: set(x) <= set(rarity))]
#     else:
#         return df

In [4]:
# def color_game_counts(s, f, c, color_filter = None):
#     df = PANDAS_CACHE[s][f][c]
#     if df.empty:
#         return (np.nan, np.nan, np.nan)
    
#     df = color_id_filter(df, color_filter)
#     df = df[['# GP','GP WR']]
#     games_played = df['# GP'].sum()
#     games_won = (df['# GP'] * (df['GP WR'] * 0.01)).sum().round()
#     percent_won = ((games_won  * 100)/ games_played).round(2)
#     return (percent_won, games_won, games_played)

# def get_color_win_rates():
#     win_rates = set_info.get_set_tree()
#     for s in SETS:
#         for f in FORMATS:
#             for c in COLOUR_GROUPS:
#                 win_rates[s][f][c] = color_game_counts(s,f,c)
#     global WINRATES
#     WINRATES = win_rates

In [5]:
# # Gets the top n cards, based on a particular stat column.
# # Can filter based on card colours, rarity.
# # Can get the bottom n cards with 'reverse=True'
# def get_top(df, stat, n=5, card_colors=None, card_rarity=None, min_thresh=1, reverse=False, columns=None):
#     if columns == None:
#         columns = list(df)
    
#     filtered = color_id_filter(df, card_colors)
#     filtered = rarity_filter(filtered, card_rarity)
#     filtered = min_play_filter(filtered, min_thresh)

#     # Functional XOR.
#     reverse = reverse != (stat == "ALSA" or stat == "ATA")
    
#     title = ""
#     # Return the smallest values if we're dealing with pick orders
#     if reverse:
#         filtered = filtered.nsmallest(n, stat)
#         #title = f"Bottom {n} cards by '{stat}'"
#     else:
#         filtered = filtered.nlargest(n, stat)
#         #title = f"Top {n} cards by '{stat}'"
    
#     # TODO: Give a clearer idea of the restrictions on the data in the DataFrame.
#     #c_filter = f"\r\nColours: '{card_colors}'" if card_colors is not None else ""
#     #r_filter = f"\r\nRarities: '{card_rarity}'" if card_colors is not None else ""
    
#     #filtered.columns.name = title + c_filter + r_filter
#     filtered.columns.name = f"'Top {n} by {stat}'"
#     filtered = filtered[columns]
#     return filtered

In [10]:
# def gen_metadata_dict():
#     ret = {
#         'Winrate' : None,
#         'Winrate Delta' : None,
#         'Games Played' : None, 
#         'Metagame Share' : None,
#         'Top Commons' : None,
#         'Top Uncommons' : None
#     }
#     return ret

# # Get metagame data for a given colour group.
# def get_color_group_metadata(s, f, colors, n=5, stat='GIH WR', columns=None):
#     # Set up the data.
#     c = WUBRG.get_color_identity(colors)
#     metagame_data = gen_metadata_dict()
    
#     # Get the frame, and handle column filtering.
#     df = PANDAS_CACHE[s][f][c]
#     if columns == None:
#         columns = list(df)
    
#     # Get the overall stats, and the stats for the colour group.
#     overall_games = color_game_counts(s, f, '')
#     color_games = color_game_counts(s, f, c)
    
#     # Use those stats to get high-level stats for the colours.
#     metagame_data['Winrate'] = color_games[0]
#     metagame_data['Winrate Delta'] = (color_games[0] - overall_games[0]).round(2)
#     metagame_data['Games Played'] = color_games[2]
#     metagame_data['Metagame Share'] = ((color_games[2] / overall_games[2]) * 100).round(2)
    
#     # Get top commons and uncommons for the colour group.
#     metagame_data['Top Commons'] = get_top(df, stat, n=n, card_rarity='C', columns=columns)
#     metagame_data['Top Uncommons'] = get_top(df, stat, n=n, card_rarity='U', columns=columns)
        
#     return metagame_data

# # Gets a suite of data about the given colour's position in the metagame.
# def get_color_overview(s, f, main_color, stat='GIH WR', columns=None):
#     # Set up the data.
#     metagame_dict = dict()
#     metagame_data = gen_metadata_dict()
#     metagame_dict[main_color] = metagame_data
    
#     # Get the frame, and handle column filtering.
#     df = PANDAS_CACHE[s][f]['']    
#     if columns == None:
#         columns = list(df)
    
#     # Get the overall stats, and the stats filtered on the main_colour.
#     overall_games = color_game_counts(s, f, '')
#     color_games = color_game_counts(s, f, '', main_color)
    
#     # Use those stats to get high-level stats for the colours.
#     metagame_data['Winrate'] = color_games[0]
#     metagame_data['Winrate Delta'] = (color_games[0] - overall_games[0]).round(2)
#     metagame_data['Games Played'] = color_games[2]
#     metagame_data['Metagame Share'] = ((color_games[2] / overall_games[2]) * 100).round(2)
    
#     # Get top commons and uncommons for the main_colour
#     metagame_data['Top Commons'] = get_top(df, stat, n=10, card_colors=main_color, card_rarity='C', columns=columns)
#     metagame_data['Top Uncommons'] = get_top(df, stat, n=10, card_colors=main_color, card_rarity='U', columns=columns)
    
#     # Get the metagame data for all colour pairs that contain 'color'.
#     color_pairs = [main_color + x for x in COLORS if x != main_color]
#     for col in color_pairs:
#         c = WUBRG.get_color_identity(col)
#         metagame_dict[c] = get_color_group_metadata(s, f, c, stat=stat, columns=columns)
        
#     # TODO: Add in a fun fact here.
        
#     return metagame_dict

# # Gets an overview of the metagame data and saves it to METAGAME_REPORT
# def get_format_metagame_data(s, f, stat='GIH WR', columns=None):
#     report = get_set_tree()
#     for s in SETS:
#         for f in FORMATS:
#             for c in COLOUR_GROUPS:
#                 report[s][f][c] = get_color_overview(s, f, c, stat=stat, columns=columns)
#     global METAGAME_REPORT
#     METAGAME_REPORT = report

In [6]:
set_info.init_cache()
DATA_CACHE = set_info.DATA_CACHE
PANDAS_CACHE = set_info.PANDAS_CACHE
WINRATES = set_info.WINRATES
METAGAME_REPORT = set_info.METAGAME_REPORT

Checking for new format data...
MID PremierDraft data is up to date!
MID TradDraft data is up to date!
Done checking for new format data.

Checking for cache updates...
Parsing MID_PremierDraft.json...
MID PremierDraft added to DATA_CACHE!
Parsing MID_TradDraft.json...
MID TradDraft added to DATA_CACHE!
Done checking for cache updates.



In [11]:
get_format_metagame_data(s, f, COLUMNS_TRUNC)
print(METAGAME_REPORT)

{'W': {'W': {'Winrate': 56.09, 'Winrate Delta': 0.35, 'Games Played': 1251807, 'Deck Share': 21.06, 'Top Commons': 'Top 10 by GIH WR'   Color Rarity   ALSA   # GP   GP WR  # GIH  GIH WR
Lunarch Veteran          W      C  4.979  59217  58.014  37502  60.821
Blessed Defiance         W      C  7.442  19949  57.888  10164  58.953
Search Party Captain     W      C  4.609  66921  57.012  44165  58.727
Gavony Trapper           W      C  5.472  64104  56.976  35203  57.717
Clarion Cathars          W      C  5.838  42055  56.312  23334  57.376
Gavony Silversmith       W      C  4.303  65337  56.487  40754  57.148
Evolving Wilds                  C  4.816  90704  56.114  50532  56.536
Candlegrove Witch        W      C  5.151  60494  56.280  39054  56.345
Mourning Patrol          W      C  5.142  59107  56.525  36304  56.319
Cathar Commando          W      C  5.431  51101  56.067  29102  56.305, 'Top Uncommons': 'Top 10 by GIH WR' Color Rarity   ALSA   # GP   GP WR  # GIH  GIH WR
Ambitious Farmhan

In [12]:
WINRATES

{'MID': {'PremierDraft': {'': (55.74, 3313478.0, 5944279),
   'W': (57.84, 7710.0, 13331),
   'U': (59.05, 7798.0, 13206),
   'B': (55.81, 3654.0, 6547),
   'R': (53.43, 5458.0, 10216),
   'G': (57.74, 6487.0, 11235),
   'WU': (58.38, 373926.0, 640523),
   'WB': (56.43, 408979.0, 724799),
   'WR': (54.06, 173120.0, 320249),
   'WG': (56.71, 302387.0, 533188),
   'UB': (58.05, 832884.0, 1434654),
   'UR': (52.01, 172425.0, 331527),
   'UG': (56.72, 239685.0, 422569),
   'BR': (54.22, 290214.0, 535221),
   'BG': (54.28, 184842.0, 340546),
   'RG': (51.17, 168174.0, 328645),
   'WUR': (46.48, 4367.0, 9395),
   'WBR': (46.58, 9587.0, 20583),
   'WBG': (49.08, 15602.0, 31787),
   'UBG': (50.33, 31165.0, 61920),
   'URG': (49.53, 10410.0, 21016),
   'WUB': (50.14, 25306.0, 50468),
   'WUG': (50.91, 12407.0, 24369),
   'WRG': (48.0, 9805.0, 20428),
   'UBR': (45.18, 6776.0, 14997),
   'BRG': (46.3, 9370.0, 20239),
   'WUBR': (0.0, 0.0, 46),
   'WUBG': (48.84, 189.0, 387),
   'WURG': (36.9, 48

In [ ]:
def format_metadata_dict(meta_dict):
    text = ''

    return text

def format_colour_overview(meta_dict):
    text = ''
    return text
    
def export_metagame_data():
    text = ''
    for c in COLORS:
        text += format_colour_overview(METAGAME_REPORT[c])
    return text

In [ ]:
s = SETS[0]
f = FORMATS[0]
metagame_info = dict()
for c in COLORS:
    metagame_info[c] = get_color_metadata(s, f, c, columns=COLUMNS_TRUNC)
    
metagame_info['U']

In [ ]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_qudrant_cards_df(df, rarity, iwd_thresh, func1, func2, cols=None):
    by_rarity = df[df['Rarity'] == rarity]
    games_played_mean = by_rarity['# GP'].mean()
    cards = by_rarity[func1(by_rarity['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    if cols is not None:
        cards = cards[cols]
    return cards


def get_trap_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, lt, gt, cols)

def get_trap_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_trap_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_niche_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, gt, lt, cols)

def get_niche_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_niche_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_staple_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, gt, gt, cols)

def get_staple_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_staple_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_dreg_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, lt, lt, cols)

def get_dreg_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_dreg_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)

In [ ]:
def filter_df(df, rarity, color):
    temp = df[df['Rarity'] == rarity]
    temp = temp[temp['Color'] == color]
    return temp

In [ ]:
df = PANDAS_CACHE['MID']['PremierDraft']['WU'].sort_values('GP WR', ascending=False)
df = df[["# GP", "GP WR", "# GIH", "GIH WR", "# GND", "GND WR"]].head(3)
print(df)

In [ ]:
premier = PANDAS_CACHE['MID']['PremierDraft']['WU'].sort_values('GP WR', ascending=False)
premier

test = premier[premier['Color'].apply(lambda x: set(x) <= set('WU'))]
#frame['Color'].apply(lambda x: any([k in x for k in 'WU']))
min_game = test['# GP'].mean() * 0.1
test[test['# GP'] >= min_game]
premier

In [ ]:
def center(df, stat):
    temp = df.copy()
    
    stat_mean = temp[stat].mean()
    temp[stat] = temp[stat] - stat_mean
    return temp.sort_values(stat, ascending=(stat == "ALSA" or stat == "ATA"))

In [ ]:
center(PANDAS_CACHE['MID']['PremierDraft']['WU'], 'GIH WR')

In [ ]:
stat = 'GIH WR'
x = get_top(PANDAS_CACHE['MID']['PremierDraft']['WU'], stat, n=5, card_colors=None, card_rarity='MR')
["Color", "ALSA", "# GP", "GP WR", "# GIH", "GIH WR"]
x

In [ ]:
s = SETS[0]
f = FORMATS[0]
stat = 'GP WR'
#get_top(PANDAS_CACHE[s][f][''], stat, n=5, card_colors=None, card_rarity=None)

rarities = ['', 'C', 'U', 'R', 'M']


def get_top_5s():
    top_5_dict = dict()
    colors = ['W', 'U', 'B', 'R', 'G']
    rarity_sets = [('C', 5), ('U', 5), ('RM', 3)]
    
    for c1 in colors:
        for c2 in colors:
            c = WUBRG.get_color_identity(c1 + c2)
            top_5_dict[c] = dict()
            for r in rarity_sets:
                color_filter = c if (len(c) == 1) else None
                top_5_dict[c][r[0]] = get_top(PANDAS_CACHE[s][f][c], stat, n=r[1], card_colors=color_filter, card_rarity=r[0])
                alias = WUBRG.get_color_alias(c)
                top_5_dict[c][r[0]].columns.name = alias
    return top_5_dict

#top_5_dict = get_top_5s()

def top_5_snippet(color, rarity):
    trgt_cols = ['ALSA', 'ATA', '# GP', 'GP WR', '# GIH', 'GIH WR']
    return top_5_dict[color][rarity][trgt_cols]

#top_5_snippet('UR', 'U')

In [ ]:
premier_commons = premier[premier['Rarity'] == 'C']
premier_commons['GP WR'].nlargest(n=5)

In [ ]:
colours = ''
rarity = 'U'
get_trap_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_niche_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_staple_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_dreg_cards('MID', 'PremierDraft', colours, rarity, -0.005)